<a href="https://colab.research.google.com/github/iavinashpathak/GhostWriter/blob/main/GhostWriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

SyntaxError: invalid syntax (__init__.py, line 1)

In [ ]:
tech_data_raw = [
    "The future of artificial intelligence is machine learning.",
    "Neural networks are the architecture of deep learning.",
    "Python remains the top language for data science.",
    "Big data and cloud computing are highly scalable."
]

In [ ]:
print(f"Total sentences: {len(tech_data_raw)}")
print(f"First sentence: {tech_data_raw[0]}")

Total sentences: 4
First sentence: The future of artificial intelligence is machine learning.


In [ ]:
tokenizer = Tokenizer(oov_token="<unk>")

In [ ]:
tokenizer.fit_on_texts(tech_data_raw)

In [ ]:
input_sequences = tokenizer.texts_to_sequences(tech_data_raw)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

print(f"Original Text: {tech_data_raw[0]}")
print(f"Tokenized Sequence: {input_sequences[0]}")
print(f"Vocabulary Size: {vocab_size}")

Original Text: The future of artificial intelligence is machine learning.
Tokenized Sequence: [2, 7, 3, 8, 9, 10, 11, 4]
Vocabulary Size: 28


In [ ]:
input_sequences_flat = []

In [ ]:
for sequence in input_sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences_flat.append(n_gram_sequence)

In [ ]:
print(f"Total N-gram sequences created: {len(input_sequences_flat)}")
print(f"Example N-gram (w1, w2, w3, w4): {input_sequences_flat[5]}")

Total N-gram sequences created: 28
Example N-gram (w1, w2, w3, w4): [2, 7, 3, 8, 9, 10, 11]


In [ ]:
max_sequence_len = max([len(x) for x in input_sequences_flat])
print(f"Max Sequence Length: {max_sequence_len}")

Max Sequence Length: 8


In [ ]:
print(f"Max Sequence Length: {max_sequence_len}")

Max Sequence Length: 8


In [ ]:
padded_sequences = pad_sequences(input_sequences_flat, maxlen=max_sequence_len, padding='pre')

In [ ]:
X = padded_sequences[:, :-1]
Y_raw = padded_sequences[:, -1]

In [ ]:
Y = to_categorical(Y_raw, num_classes=vocab_size)

In [ ]:
print("-" * 30)
print("Data Preparation Summary:")
print(f"Padded Input Shape (X): {X.shape}")
print(f"Target Output Shape (Y): {Y.shape}")
print(f"Example X (Input sequence): {X[5]}")
print(f"Example Y (Target one-hot vector): {Y[5]}")

------------------------------
Data Preparation Summary:
Padded Input Shape (X): (28, 7)
Target Output Shape (Y): (28, 28)
Example X (Input sequence): [ 0  2  7  3  8  9 10]
Example Y (Target one-hot vector): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
EMBEDDING_DIM = 100
LSTM_UNITS = 128
INPUT_LENGTH = X.shape[1]

In [ ]:
model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=INPUT_LENGTH))

# Stacked LSTM Layers
model.add(LSTM(LSTM_UNITS, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(LSTM_UNITS))
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(vocab_size, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X, Y, epochs=50, verbose=1, batch_size=32)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0357 - loss: 3.3330
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.0714 - loss: 3.3273
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0357 - loss: 3.3231
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.1071 - loss: 3.3158
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1786 - loss: 3.3086
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2143 - loss: 3.3030
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.1786 - loss: 3.2955
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2143 - loss: 3.2848
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2500 - loss: 3.2670
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2143 - loss: 3.2463
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1429 - loss: 3.2201
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2143 - loss: 3.1955
Ep

In [ ]:
final_accuracy = history.history['accuracy'][-1]
print(f"\nFinal Training Accuracy: {final_accuracy:.4f}")


Final Training Accuracy: 0.4643


In [ ]:
import pickle

In [ ]:
model_file_path = 'model.h5'
model.save(model_file_path)
print(f"Model successfully saved to {model_file_path}")

Model successfully saved to model.h5


In [ ]:
tokenizer_file_path = 'tokenizer.pickle'
with open(tokenizer_file_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Tokenizer successfully saved to {tokenizer_file_path}")

Tokenizer successfully saved to tokenizer.pickle
